In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import requests


In [6]:
import pymysql
import mysql.connector as mysql

# database connection
db = mysql.connect(user = "root", database = "hadith")
cursor = db.cursor()


In [8]:
from bs4 import BeautifulSoup

response = requests.get('https://40hadithnawawi.com/')
soup = BeautifulSoup(response.content, 'html.parser')
books = soup.find_all('div', class_='book')

for book in books:
    book_name = book.find('h2').text
    book_description = book.find('p').text
    book_image = book.find('img')['src']
    book_link = book.find('a')['href']
    print(book_name)
    print(book_description)
    print(book_image)
    print(book_link)
    print('------------------------')

    # insert into database
    cursor.execute("INSERT INTO books (name, description, image, link) VALUES (%s, %s, %s, %s)", (book_name, book_description, book_image, book_link))
    cursor.commit()

    # get the last inserted id
    book_id = cursor.lastrowid

    # get the book details
    response = requests.get(book_link)
    soup = BeautifulSoup(response.content, 'html.parser')
    chapters = soup.find_all('div', class_='chapter')

    for chapter in chapters:
        chapter_name = chapter.find('h3').text
        chapter_description = chapter.find('p').text
        chapter_link = chapter.find('a')['href']
        print(chapter_name)
        print(chapter_description)
        print(chapter_link)
        print('------------------------')

        # insert into database
        cursor.execute("INSERT INTO chapters (name, description, link, book_id) VALUES (%s, %s, %s, %s)", (chapter_name, chapter_description, chapter_link, book_id))
        cursor.commit()

        # get the last inserted id
        chapter_id = cursor.lastrowid

        # get the chapter details
        response = requests.get(chapter_link)
        soup = BeautifulSoup(response.content, 'html.parser')
        hadiths = soup.find_all('div', class_='hadith')

        for hadith in hadiths:
            hadith_number = hadith.find('h4').text
            hadith_text = hadith.find('p').text
            print(hadith_number)
            print(hadith_text)
            print('------------------------')

            # insert into database
            cursor.execute("INSERT INTO hadiths (number, text, chapter_id) VALUES (%s, %s, %s)", (hadith_number, hadith_text, chapter_id))
            cursor.commit()
            

# close the connection
cursor.close()
            


True